# MNIST project

Training a Neural Network Model for recognizing the numbers.

Author: AsalDelkhosh
Year: 2022

This is a project about using non-negative matrix factorization for weight initialization in autoencoders.

Autoencoders are a specific form of neural networks build by two parts. First part is used to encode the input data to a space with lower dimension and the second part decodes the abstracted data to reconstruct the input.

The networks is trained in a way that the reocnstructed data is very similar to the original input. This way we can make sure that we are transferring enough information to a small encoded space so that we are enable to reconstruct the entire image from it.

## 1.import

First we need to import some modules and libraries like numpy, tensorflow, keras and matplotlib.
This libraries will help us to build our model and train it.

In [4]:
# removing warnings
import warnings
warnings.filterwarnings('ignore')

In [5]:
# displaying HTML
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [6]:
# tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist, fashion_mnist
from tensorflow.keras.callbacks import TensorBoard, History
# sklearn
from sklearn.decomposition import NMF
# numpy
import numpy as np
# matplotlib
import matplotlib.pyplot as plt
# coounter
from collections import Counter

ModuleNotFoundError: No module named 'tensorflow'

## 2.load data 

In the next part, we are going to load our databset into train data and train labels.

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

### Data normalizing

In this part we are going to normilize our dataset.

In [ ]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# reshaping our data
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

### Selecting

In this part we are going to select some of our samples.

In [ ]:
print(type(x_test))

selected_samples = np.zeros(10)
l = []
i = 0
while len(l)<10:
    if y_test[i] not in l:
        l.append(y_test[i])
        selected_samples[y_test[i]] = int(i)
        i = i + 1
    else:
        i = i + 1
selected_samples = list(selected_samples)
selected_samples = [int(s) for s in selected_samples]

## Functions

Our main functions for creating auto encoders and displaying the output images.

### Displayin images

In [ ]:
def display_images(x_test, decoded_imgs, selected_samples, image_type=[], name='reconstruction'):    
    n = len(selected_samples)  # how many digits we will display
    plt.figure(figsize=(14, 8))
    plt.title('Comparison of original and reconstructed image using different initialization methods')
    image_type = ['Original Image']+image_type
    for j in range(len(image_type)):
        ax = plt.subplot(len(image_type), n+2, (j*(n+2)+1))
        ax.text(0, 0.5, image_type[j], fontsize=16)
        ax.axis("off")
        
    for i in range(n):

        # display original
        ax = plt.subplot(len(image_type), n+2, i + 3)
        plt.imshow(x_test[selected_samples[i]].reshape(28, 28))
        plt.gray()
        ax.axis("off")

        for j in range(1,len(image_type)):
            # display reconstruction with different initialization
            ax = plt.subplot(len(image_type), n+2, i + 3 + (j)*(n+2))
            plt.imshow(decoded_imgs[image_type[j]][selected_samples[i]].reshape(28, 28))
            plt.gray()
            ax.axis("off")
    plt.tight_layout()        
    plt.subplots_adjust(hspace=0.05, wspace=0.05)
    plt.savefig(name+'.png', dpi=200)
    plt.show()

### AutoEncoder

Creating autoencoder based on our recognizing problem.

In [ ]:
def build_autoencoder(initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)):
    
    if initializer!='nmf':
        # inp
        input_img = Input(shape=(784,))
        encoded = Dense(128, activation='relu', kernel_initializer=initializer)(input_img)
        encoded = Dense(64, activation='relu', kernel_initializer=initializer)(encoded)
        encoded = Dense(32, activation='relu', kernel_initializer=initializer)(encoded)

        decoded = Dense(64, activation='relu', kernel_initializer=initializer)(encoded)
        decoded = Dense(128, activation='relu', kernel_initializer=initializer)(decoded)
        decoded = Dense(784, activation='sigmoid', kernel_initializer=initializer)(decoded)
        autoencoder = Model(input_img, decoded)
        encoder = Model(input_img, encoded)
        
    else:
        input_img = Input(shape=(784,))
        encoded = Dense(128, activation='relu')(input_img)
        encoded = Dense(64, activation='relu')(encoded)
        encoded = Dense(32, activation='relu')(encoded)
        
        decoded = Dense(64, activation='relu')(encoded)
        decoded = Dense(128, activation='relu')(decoded)
        decoded = Dense(784, activation='sigmoid')(decoded)
        
        autoencoder = Model(input_img, decoded)
        encoder = Model(input_img, encoded)
    
        init_weights = get_nmf_init_weights([784, 128, 64, 32])
        autoencoder.set_weights(init_weights)
    
    autoencoder.compile(optimizer='adam', loss='mse')
#     autoencoder.summary()
    
    return autoencoder

### Defining our methods

In [ ]:
methods = ['RandomNormal', 'GlorotNormal']
small_sample_size = 3000

## Training our model

In the next part we are going to build our model and train our model
with the dataset that we loaded.

In [ ]:
# hyper parameters
sample_size='all'
epochs = 50

# auto encoders
autoencoders = {}
autoencoders['RandomNormal'] = build_autoencoder(initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None))
autoencoders['GlorotNormal'] = build_autoencoder(initializer=tf.keras.initializers.GlorotNormal())


# hist dictionary
hist = {}
for init_method in methods:
    print("Training the model with ", init_method, " initializer...")
    hist[init_method] = History()
    # fitting our auto encoder
    autoencoders[init_method].fit(x_train, x_train,
                                  epochs=epochs,
                                  batch_size=256,
                                  shuffle=True,
                                  validation_split=0.33,
                                  callbacks=[hist[init_method]],
                                  verbose=0)
    
# decoding images
decoded_imgs = {}
for init_method in methods:
    decoded_imgs[init_method] = autoencoders[init_method].predict(x_test)
    
# displaying images
display_images(x_test, decoded_imgs, selected_samples, image_type=methods, name='images/mnist_digit_reconstruction_'+str(sample_size)+'_'+str(epochs))